# Linear Optimization with PuLP

First we need to install the package!

In [24]:
try:
  from pulp import *
except:
  !pip install pulp
  from pulp import *

import pandas as pd

## Our first example problem

The first step in using PuLP is to instantiate the underlying problem class, i.e. create an empty model which will then be populated - the only model we specify here is the optiization sense (LpMaximize or LpMinimize). Note that this can already be printed (of course it is quite boring):


In [25]:
model = LpProblem('my_bakery_problem', LpMaximize)
print(model)

my_bakery_problem:
MAXIMIZE
None
VARIABLES



Following our three-step-approach we next want to create decision variables. The syntax facilitates the direct specification of lower and upper bounds.

In [26]:
# Create decision variables
A = LpVariable('A', lowBound=0)
B = LpVariable('B', lowBound=0)
print(model)

my_bakery_problem:
MAXIMIZE
None
VARIABLES



Note that we create references to LpVariable objects which we can subsequently call in objective function and constraints.

Our objective function is simply added onto the model and we can print the problem again to verify if we were successful:

In [27]:
# Objective function
model += 20 * A + 40 * B, "Profit"
print(model)

my_bakery_problem:
MAXIMIZE
20*A + 40*B + 0
VARIABLES
A Continuous
B Continuous



Unconstrained optimization is of course boring - so we just add our constraints onto the model object:

In [28]:
# Constraints
model += 0.5 * A + 1 * B <= 30
model += 1 * A + 2.5 * B <= 60
model += 1 * A + 2 * B <= 22

print(model)

my_bakery_problem:
MAXIMIZE
20*A + 40*B + 0
SUBJECT TO
_C1: 0.5 A + B <= 30

_C2: A + 2.5 B <= 60

_C3: A + 2 B <= 22

VARIABLES
A Continuous
B Continuous



Note that the last output was not so nice as we did not name our constraints and in turn receive generic constraint names like _C1. Let's recreate the complete optimization problem in one go and introduce meaningful names:

In [29]:
# Base model
model = LpProblem('my_bakery_problem', LpMaximize)
# Create decision variables
A = LpVariable('A', lowBound=0)
B = LpVariable('B', lowBound=0)
# Objective function
model += 20 * A + 40 * B, "Profit"
# Constraints
model += 0.5 * A + 1 * B <= 30, "Oven Availability (1x30)"
model += 1 * A + 2.5 * B <= 60, "Baker Availability (2x30)"
model += 1 * A + 2 * B <= 22, "Packer Availability (1x22)"

print(model)

my_bakery_problem:
MAXIMIZE
20*A + 40*B + 0
SUBJECT TO
Oven_Availability_(1x30): 0.5 A + B <= 30

Baker_Availability_(2x30): A + 2.5 B <= 60

Packer_Availability_(1x22): A + 2 B <= 22

VARIABLES
A Continuous
B Continuous



Much better - with the modeling successfully completed we next proceed to solving the problem which is achieved by simply calling the model object's solve function:

In [30]:
# Solve our problem
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2823141b10a448ed9479a295fa93a8ca-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/2823141b10a448ed9479a295fa93a8ca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
0  Obj -0 Dual inf 80 (2)
0  Obj -0 Dual inf 80 (2)
1  Obj 440
Optimal - objective value 440
Optimal objective 440 - 1 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



1

What is 1? Not the result we were expecting but of course this is just a status code which we can also access directly:

In [31]:
model.status

1

What does 1 mean in this case? The pulp import establishes the following dictionary whith status code meanings:

In [32]:
LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

Then we can easily verify that we optimally solved the problem by calling the following:

In [33]:
LpStatus[model.status]

'Optimal'

How to we access this optimal solution? Remember that we created references to our decision variables in the beginning. It turns out it is these variables which were then referenced by the model that carry their optimal value assignment in the property `varValue`:

In [34]:
# Print our decision variable values
print(f"Optimal production amount of Cake A = {A.varValue}")
print(f"Optimal production amount of Cake B = {B.varValue}")

Optimal production amount of Cake A = 0.0
Optimal production amount of Cake B = 11.0


Similarly, the corresponding objective value can be accessed via `model.objective`:

In [35]:
# Print our objective function value
print(f"Optimal profit is {model.objective.value()}")


Optimal profit is 440.0


This was our first example - not too bad I would say!



### Modeling the complex bakery

In [36]:
# Cake Database
cake_data	=	{   "A": {'profit': 20, 'oven' : 0.5, 'bakers' : 1, 'packers' : 1, 'orders' : 1},
                    "B": {'profit': 40, 'oven' : 1, 'bakers' : 2.5, 'packers' : 2, 'orders' : 2},
                    "C": {'profit': 33, 'oven' : 1, 'bakers' : 2, 'packers' : 2, 'orders' : 0},
                    "D": {'profit': 14, 'oven' : 0.5, 'bakers' : 1.5, 'packers' : 1, 'orders' : 1},
                    "E": {'profit': 6, 'oven' : 0.25, 'bakers' : 0.25, 'packers' : 1, 'orders' : 1},
                    "F": {'profit': 60, 'oven' : 2.5, 'bakers' : 3, 'packers' : 3, 'orders' : 0}}

In [37]:
#Model and Variables
model = LpProblem("My_complex_bakery", LpMaximize)
cakeDecisions = LpVariable.dict('make', cake_data, 0)
cakeDecisions

{'A': make_A, 'B': make_B, 'C': make_C, 'D': make_D, 'E': make_E, 'F': make_F}

In [38]:
#Objective Function
model += lpSum([cakeDecisions[i] * cake_data[i]['profit'] for i in cake_data.keys()])
print(model)

My_complex_bakery:
MAXIMIZE
20*make_A + 40*make_B + 33*make_C + 14*make_D + 6*make_E + 60*make_F + 0
VARIABLES
make_A Continuous
make_B Continuous
make_C Continuous
make_D Continuous
make_E Continuous
make_F Continuous



In [39]:
#Constraints and Solve
model += lpSum([cakeDecisions[i] * cake_data[i]['oven'] for i in cake_data.keys()]) <= 30, "oven capacity"
model += lpSum([cakeDecisions[i] * cake_data[i]['bakers'] for i in cake_data.keys()]) <= 60, "baker capacity"
model += lpSum([cakeDecisions[i] * cake_data[i]['packers'] for i in cake_data.keys()]) <= 22, "packer capacity"

for i in cake_data.keys():
    model += cakeDecisions[i] >= cake_data[i]['orders'], "orders "+i

print(model)


My_complex_bakery:
MAXIMIZE
20*make_A + 40*make_B + 33*make_C + 14*make_D + 6*make_E + 60*make_F + 0
SUBJECT TO
oven_capacity: 0.5 make_A + make_B + make_C + 0.5 make_D + 0.25 make_E
 + 2.5 make_F <= 30

baker_capacity: make_A + 2.5 make_B + 2 make_C + 1.5 make_D + 0.25 make_E
 + 3 make_F <= 60

packer_capacity: make_A + 2 make_B + 2 make_C + make_D + make_E + 3 make_F
 <= 22

orders_A: make_A >= 1

orders_B: make_B >= 2

orders_C: make_C >= 0

orders_D: make_D >= 1

orders_E: make_E >= 1

orders_F: make_F >= 0

VARIABLES
make_A Continuous
make_B Continuous
make_C Continuous
make_D Continuous
make_E Continuous
make_F Continuous



In [40]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e8d154bf367549b6ab88aeede59eaff4-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/e8d154bf367549b6ab88aeede59eaff4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 45 RHS
At line 55 BOUNDS
At line 56 ENDATA
Problem MODEL has 9 rows, 6 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-6) rows, 6 (0) columns and 18 (-6) elements
0  Obj 120 Dual inf 263.49999 (6)
3  Obj 420
Optimal - objective value 420
After Postsolve, objective 420, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 420 - 3 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



1

In [41]:
for i in cakeDecisions:
    print("Cake " + str(i) + ": " + str(cakeDecisions[i].value()))

Cake A: 1.0
Cake B: 9.5
Cake C: 0.0
Cake D: 1.0
Cake E: 1.0
Cake F: 0.0


In [42]:
model.objective.value()

420.0

## Example 2 - Investment Problem

We want to invest in bonds and have indentified two attractive options:

Bond A:
 * Yield: 4 (percent)
 * Risk rating: 2 (higher is worse)
 * Maturity: 3 (years)

Bond B:
 * Yield: 3
 * Risk rating: 1
 * Maturity: 4

We can invest up to 100$. We want a weighted risk score of at most 1.5 for our portfolio. Furthermore, the average maturity should not exceed 3.6.

__Try to accomplish the following things:__
* Create and solve the optimization problem analogue to the prior problem
* Create a simple sensitivity analysis for the bond investment problem. The analysis should systematically change one of the two risk parameters, optimize the portfolio for each setting and store the solution in a dictionary.
* BONUS: can you plot the one-way parameter change (You can use `from_dict(results, orient='index')` to obtain a dataframe)

In [43]:
bonds = {"A": {"risk": 2, "yield" : 4, "maturity" : 3},
         "B": {"risk": 1, "yield" : 3, "maturity" : 4}}

bond_model = LpProblem("my-bond-problem", LpMaximize)

X = LpVariable.dict("buy",bonds,0)

bond_model += lpSum([X[i] * bonds[i]['yield'] / 100 for i in bonds.keys()])

bond_model += lpSum([X[i] * bonds[i]['risk']] for i in bonds.keys()) <= 1.5 * lpSum(X[i] for i in bonds.keys()), "risk"
bond_model += lpSum([X[i] for i in bonds.keys()]) <= 100, "budget"
bond_model += lpSum([X[i] * bonds[i]['maturity'] for i in bonds.keys()]) <= 3.6 * lpSum([X[i] for i in bonds.keys()]), "maturity"

print(bond_model)

my-bond-problem:
MAXIMIZE
0.04*buy_A + 0.03*buy_B + 0.0
SUBJECT TO
risk: 0.5 buy_A - 0.5 buy_B <= 0

budget: buy_A + buy_B <= 100

maturity: - 0.6 buy_A + 0.4 buy_B <= 0

VARIABLES
buy_A Continuous
buy_B Continuous



In [44]:
bond_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/85031186a1014060a642d69eff2bdf62-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/85031186a1014060a642d69eff2bdf62-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
0  Obj -0 Dual inf 0.0699998 (2)
0  Obj -0 Dual inf 0.0699998 (2)
2  Obj 3.5
Optimal - objective value 3.5
Optimal objective 3.5 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



1

In [45]:
for i in X:
    print(i, X[i].varValue)

print(bond_model.objective.value())

A 50.0
B 50.0
3.5


In [46]:
riskValues = list(range(0,200))
results = []
for bRisk in riskValues:
    bonds["B"]["risk"] = bRisk / 100
    bond_model = LpProblem("my-bond-problem", LpMaximize)
    X = LpVariable.dict("buy",bonds,0)
    bond_model += lpSum([X[i] * bonds[i]['yield'] / 100 for i in bonds.keys()])
    bond_model += lpSum([X[i] * bonds[i]['risk']] for i in bonds.keys()) <= 1.5 * 100, "risk"
    bond_model += lpSum([X[i] for i in bonds.keys()]) <= 100, "budget"
    bond_model += lpSum([X[i] * bonds[i]['maturity'] for i in bonds.keys()]) <= 4 * 100, "maturity"
    bond_model.solve(PULP_CBC_CMD(msg=0))
    results.append({'b_risk' : bRisk / 100, 'value' : bond_model.objective.value(), 'A' : X['A'].varValue, 'B':X['B'].varValue})

results = pd.DataFrame(results)

In [47]:
pip install lets_plot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.3 MB/s eta 0:00:00eta 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
from lets_plot import *
import pandas as pd
LetsPlot.setup_html()

results['risk'] = (results['A'] * bonds['A']['risk'] + results['B'] * results['b_risk']) / 100
results['investment'] = results['A'] + results['B']
ggplot(results,aes(x='b_risk')) + \
geom_line(aes(y="A"), color = "blue", manual_key="A") + \
geom_line(aes(y="B"), color = "green", manual_key="B") + \
geom_line(aes(y="investment"), color = "black", manual_key="Total")

In [49]:
ggplot(pd.DataFrame(results),aes(x='b_risk')) + \
geom_line(aes(y="value"), color = "red", manual_key="value") + \
geom_line(aes(y="risk"), color = "orange", manual_key="risk score")

## Example 3 - Tie Manufacturing
Fifth Avenue Industries produces 4 types of ties
* Production is subject to minimum and maximum output constraints
* Production requires 3 different types of materials
* Material storage capacity is limited
* Decision problem: “How many ties of each type should the company produce?”
  * Objective → Profit maximization

### Material cost and availability

Material | Cost per yard | Yards available
---|---|---
Silk|20|1000
Polyester|6|2000
Cotton|9|1250


### Different tie production options

Type | Price | Monthly Minimum | Monthly Maximum
---|---|---|---
Silk|5.95|6000|7000
Polyester| 2.80|10000|14000
Blend 1| 3.50|13000|16000
Blend 2| 4.00|6000|8500

### Material consumption per tie


Material|Silk|Polyester|Blend 1|Blend 2
---|---|---|---|---
Silk|0.125|0|0|0
Polyester|0|0.08|0.05|0.03
Cotton|0|0|0.05|0.07


This problem points us to a situation where setting up the problem as before can get easily difficult to handle. Therefore, we want to lists (to concisely address repetitive data structures) and list comprehensions (to automatically create constraints).

First we replicate the input tables as dicts of dicts:

In [50]:
model = LpProblem('tie_production', LpMaximize)

In [51]:
materials = {"silk" : {"cost" : 20, "availability" : 1000},
             "polyester" : {"cost" : 6, "availability" : 2000},
             "cotton" : {"cost" : 9, "availability" : 1250}}

In [52]:
products = {"Silk-Tie" : {"price" : 5.95, "prod-min" : 6000, "prod-max" : 7000, "silk" : 0.125, "polyester" : 0, "cotton" : 0},
        "Polyester-Tie" : {"price" : 2.80, "prod-min" : 10000, "prod-max" : 14000, "silk" : 0, "polyester" : 0.08, "cotton" : 0},
        "Blend-1-Tie" : {"price" : 3.50, "prod-min" : 13000, "prod-max" : 16000, "silk" : 0, "polyester" : 0.05, "cotton" : 0.05},
        "Blend-2-Tie" : {"price" : 4.00, "prod-min" : 6000, "prod-max" : 8500, "silk" : 0, "polyester" : 0.03, "cotton" : 0.07},
          "Blend-100-Tie" : {"price" : 100.00, "prod-min" : 0, "prod-max" : 200, "silk" : 0.2, "polyester" : 0.01, "cotton" : 0.01}}

In [53]:
print(materials)
print(products)

{'silk': {'cost': 20, 'availability': 1000}, 'polyester': {'cost': 6, 'availability': 2000}, 'cotton': {'cost': 9, 'availability': 1250}}
{'Silk-Tie': {'price': 5.95, 'prod-min': 6000, 'prod-max': 7000, 'silk': 0.125, 'polyester': 0, 'cotton': 0}, 'Polyester-Tie': {'price': 2.8, 'prod-min': 10000, 'prod-max': 14000, 'silk': 0, 'polyester': 0.08, 'cotton': 0}, 'Blend-1-Tie': {'price': 3.5, 'prod-min': 13000, 'prod-max': 16000, 'silk': 0, 'polyester': 0.05, 'cotton': 0.05}, 'Blend-2-Tie': {'price': 4.0, 'prod-min': 6000, 'prod-max': 8500, 'silk': 0, 'polyester': 0.03, 'cotton': 0.07}, 'Blend-100-Tie': {'price': 100.0, 'prod-min': 0, 'prod-max': 200, 'silk': 0.2, 'polyester': 0.01, 'cotton': 0.01}}


Now this seems somewhat work-intesive in this demonstration setting. But note that this data would typically come from an operational system specifying the costs, warehouse levels, demand and margins.

Note that coming from pandas dataframes you can easily retrieve dictionaries using `pd.DataFrame.to_dict`.

We next want to create the decision variables. Note that we want to create one per entry in the ties dict. PuLP offers the `LpVariable.dicts` function this common scenario. The function creates a dictionary of decision variables by specifying a common prefix (first argument), a list of the future keys (second argument) plus the upper and lower bound. We choose "amount" to be our prefix and use the keys from our products dict. We do not specify the lower and upper bounds as they will depend on the product:

In [54]:
products.keys()

dict_keys(['Silk-Tie', 'Polyester-Tie', 'Blend-1-Tie', 'Blend-2-Tie', 'Blend-100-Tie'])

In [55]:
amounts = LpVariable.dicts('amount', products.keys(), 0)

In [56]:
amounts

{'Silk-Tie': amount_Silk_Tie,
 'Polyester-Tie': amount_Polyester_Tie,
 'Blend-1-Tie': amount_Blend_1_Tie,
 'Blend-2-Tie': amount_Blend_2_Tie,
 'Blend-100-Tie': amount_Blend_100_Tie}

We can now proceed setting up our optimization problem. Unlike before we want to first consider the constraints. There were two types:
* production amounts need to fulfill min and max as specified in products
* usage of any material must not exceeds its availability

Let's first tackle the upper and lower bounds:



In [57]:
for p in products: #iterate over dict returns keys
  print(p)

Silk-Tie
Polyester-Tie
Blend-1-Tie
Blend-2-Tie
Blend-100-Tie


In [58]:
for p in products: #iterate over all products
  model += amounts[p] >= products[p]["prod-min"], f"minimum production of {p}"
  model += amounts[p] <= products[p]["prod-max"], f"maximum production of {p}"

print(model)

tie_production:
MAXIMIZE
None
SUBJECT TO
minimum_production_of_Silk_Tie: amount_Silk_Tie >= 6000

maximum_production_of_Silk_Tie: amount_Silk_Tie <= 7000

minimum_production_of_Polyester_Tie: amount_Polyester_Tie >= 10000

maximum_production_of_Polyester_Tie: amount_Polyester_Tie <= 14000

minimum_production_of_Blend_1_Tie: amount_Blend_1_Tie >= 13000

maximum_production_of_Blend_1_Tie: amount_Blend_1_Tie <= 16000

minimum_production_of_Blend_2_Tie: amount_Blend_2_Tie >= 6000

maximum_production_of_Blend_2_Tie: amount_Blend_2_Tie <= 8500

minimum_production_of_Blend_100_Tie: amount_Blend_100_Tie >= 0

maximum_production_of_Blend_100_Tie: amount_Blend_100_Tie <= 200

VARIABLES
amount_Blend_100_Tie Continuous
amount_Blend_1_Tie Continuous
amount_Blend_2_Tie Continuous
amount_Polyester_Tie Continuous
amount_Silk_Tie Continuous



Note how the lower part of the model still states that the the variables are unconstrained (free) as the min and max amounts are established via constraints. A more conscise way would have been modifying the `lowBound` and `upBound` properties of the production amount variables:

In [59]:
model = LpProblem('tie_production', LpMaximize) #reset the model

for p in products: #iterate over all products
  amounts[p].lowBound = products[p]["prod-min"]
  amounts[p].upBound = products[p]["prod-max"]

In [60]:
print(model)

tie_production:
MAXIMIZE
None
VARIABLES



The second set of constraints refers to the production materials. Therefore we must iterate over all products and for each product determine the total production amount across the products. The automated evaluation of objects created and evaluated at runtime is done via `lpSum` objects which instantiates a linear combination over a list. Lets first print out these list which we create using list comprehensions:

In [61]:
for m in materials:
  print(list(amounts[p] * products[p][m] for p in products))

[0.125*amount_Silk_Tie + 0.0, 0, 0, 0, 0.2*amount_Blend_100_Tie + 0.0]
[0, 0.08*amount_Polyester_Tie + 0.0, 0.05*amount_Blend_1_Tie + 0.0, 0.03*amount_Blend_2_Tie + 0.0, 0.01*amount_Blend_100_Tie + 0.0]
[0, 0, 0.05*amount_Blend_1_Tie + 0.0, 0.07*amount_Blend_2_Tie + 0.0, 0.01*amount_Blend_100_Tie + 0.0]


This looks good - we can wrap the list comprehension output using `lpSum` to sum over them, include the right hand side and incorporate the constraints in our model:

In [62]:
for m in materials:
  model += lpSum(amounts[p] * products[p][m] for p in products) <= materials[m]["availability"], f"{m} availability"

print(model)

tie_production:
MAXIMIZE
None
SUBJECT TO
silk_availability: 0.2 amount_Blend_100_Tie + 0.125 amount_Silk_Tie <= 1000

polyester_availability: 0.01 amount_Blend_100_Tie + 0.05 amount_Blend_1_Tie
 + 0.03 amount_Blend_2_Tie + 0.08 amount_Polyester_Tie <= 2000

cotton_availability: 0.01 amount_Blend_100_Tie + 0.05 amount_Blend_1_Tie
 + 0.07 amount_Blend_2_Tie <= 1250

VARIABLES
amount_Blend_100_Tie <= 200 Continuous
13000 <= amount_Blend_1_Tie <= 16000 Continuous
6000 <= amount_Blend_2_Tie <= 8500 Continuous
10000 <= amount_Polyester_Tie <= 14000 Continuous
6000 <= amount_Silk_Tie <= 7000 Continuous



We can now wrap up by setting up the objective function which we want to iterate over all the ties and evaluate as produced amount * (sales price - material cost). Note that each instance of material cost necessitates an lpSum over all materials while the whole objective function is an lpSum over all products. We end up with a double sum which manifests as nested lpSum objects:

In [63]:
model += lpSum((amounts[p] * (products[p]["price"] - lpSum(products[p][m] * materials[m]["cost"] for m in materials))) for p in products)

In [64]:
print(model)

tie_production:
MAXIMIZE
95.85*amount_Blend_100_Tie + 2.75*amount_Blend_1_Tie + 3.19*amount_Blend_2_Tie + 2.32*amount_Polyester_Tie + 3.45*amount_Silk_Tie + 0.0
SUBJECT TO
silk_availability: 0.2 amount_Blend_100_Tie + 0.125 amount_Silk_Tie <= 1000

polyester_availability: 0.01 amount_Blend_100_Tie + 0.05 amount_Blend_1_Tie
 + 0.03 amount_Blend_2_Tie + 0.08 amount_Polyester_Tie <= 2000

cotton_availability: 0.01 amount_Blend_100_Tie + 0.05 amount_Blend_1_Tie
 + 0.07 amount_Blend_2_Tie <= 1250

VARIABLES
amount_Blend_100_Tie <= 200 Continuous
13000 <= amount_Blend_1_Tie <= 16000 Continuous
6000 <= amount_Blend_2_Tie <= 8500 Continuous
10000 <= amount_Polyester_Tie <= 14000 Continuous
6000 <= amount_Silk_Tie <= 7000 Continuous



In [65]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/77ad241978d0481ab7693c3a2cfcdbe0-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/77ad241978d0481ab7693c3a2cfcdbe0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 23 RHS
At line 27 BOUNDS
At line 37 ENDATA
Problem MODEL has 3 rows, 5 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-1) rows, 4 (-1) columns and 7 (-2) elements
0  Obj 102240 Dual inf 680.31 (4)
2  Obj 137960
Optimal - objective value 137960
After Postsolve, objective 137960, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 137960 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):   

1

Having successful solved the problem we want to inspect the optimal production program and the optimal profit:

In [66]:
for p in products:
  print(p,":",amounts[p].varValue)

Silk-Tie : 7000.0
Polyester-Tie : 13625.0
Blend-1-Tie : 13060.0
Blend-2-Tie : 8500.0
Blend-100-Tie : 200.0


In [67]:
#print("Profit:", model.objective)
model.objective.value()

137960.0

It's instructive to inspect the usage of the resouces. To this end we print out the slack and the shadow prices of the constraints:

In [68]:
for con in model.constraints:
  print(con, "Slack:", model.constraints[con].slack, "Shadow Price:", model.constraints[con].pi)

silk_availability Slack: 85.0 Shadow Price: -0.0
polyester_availability Slack: -0.0 Shadow Price: 29.0
cotton_availability Slack: -0.0 Shadow Price: 26.0


## Example 4 - Marketing Mix

Solve the following marketing mix problem using the dictionary modeling approach. What do you make of the solution?

Win Big Gambling Club wants to optimize its marketing mix
\$8000 are available for advertisement
* TV spots
* Newspaper ad
* Prime time radio spots (30 seconds at 8pm)
* Shoulder time radio spots (1 min. in the afternoon)

Decision problem:
* How should the marketing mix be designed?
* Objective → Maximize reach

Requirements:
* At least 5 radio spots
* At most \$1.800 for radio

The problem information is provided in the following dictionaries which specify cost per spot, reach, and maximum availability:

In [69]:
options = {"tv": {"reach" : 5000, "cost" : 800, "max" : 12},
           "newspaper" : {"reach" : 8500, "cost" : 925, "max" : 5},
           "radio-prime": {"reach" : 2400, "cost" : 290, "max" : 25},
           "radio-shoulder": {"reach" : 2800, "cost" : 380, "max" : 20}}

In [70]:
numberofspots = LpVariable.dicts('number', options.keys(), 0)

In [71]:
for i in numberofspots.keys():
    numberofspots[i].upBound = options[i]["max"]

In [72]:
numberofspots["tv"]

number_tv

In [73]:
reachProblem = LpProblem("marketing-mix", LpMaximize)
reachProblem += lpSum([numberofspots[i]*options[i]['reach'] for i in options.keys()])
reachProblem += numberofspots['radio-prime'] + numberofspots['radio-shoulder'] >= 5, "radio requirement"
reachProblem += numberofspots['radio-prime'] * options['radio-prime']['cost'] + numberofspots['radio-shoulder']* options['radio-shoulder']['cost'] <= 1800, "radio limit"
print(reachProblem)

marketing-mix:
MAXIMIZE
8500*number_newspaper + 2400*number_radio_prime + 2800*number_radio_shoulder + 5000*number_tv + 0
SUBJECT TO
radio_requirement: number_radio_prime + number_radio_shoulder >= 5

radio_limit: 290 number_radio_prime + 380 number_radio_shoulder <= 1800

VARIABLES
number_newspaper <= 5 Continuous
number_radio_prime <= 25 Continuous
number_radio_shoulder <= 20 Continuous
number_tv <= 12 Continuous



In [74]:
reachProblem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3d7fc129a2604befa39fb75f4a0a36dc-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/3d7fc129a2604befa39fb75f4a0a36dc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 16 RHS
At line 19 BOUNDS
At line 24 ENDATA
Problem MODEL has 2 rows, 4 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (0) rows, 2 (-2) columns and 4 (0) elements
0  Obj 104692.51 Primal inf 4.0864517 (1) Dual inf 5200 (2)
1  Obj 117396.55
Optimal - objective value 117396.55
After Postsolve, objective 117396.55, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 117396.5517 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds)

1

In [75]:
for i in numberofspots.keys():
    print(i, numberofspots[i].varValue)

tv 12.0
newspaper 5.0
radio-prime 6.2068966
radio-shoulder 0.0


## Example 5 (Complex Investment Problem)

| Investment           | Interest Rate | Risk Score |
|----------------------|---------------|------------|
| Trade credits        | 7%            | 1.7        |
| Corp. bonds          | 10%           | 1.2        |
| Gold stocks          | 19%           | 3.7        |
| Platinum stocks      | 12%           | 2.4        |
| Mortgage securities  | 8%            | 2.0        |
| Construction loans   | 14%           | 2.9        |

* Up to $ 5 million can be invested
* At most 25% per investment type
* At least 30% in Gold and Platinum
* At least 45% in trade credits and corporate bonds

* Total risk must not exceed 2.0  - Total risk = Risk score of individual asset weighted by investment share


In [76]:
bonds = {"Gold Stocks": {"risk": 3.7, "yield" : 0.19},
         "Platinum Stocks": {"risk": 2.4, "yield" : 0.12},
         "Mortgage Sec": {"risk": 2, "yield" : 0.08},
         "Trade Credits": {"risk": 1.7, "yield" : 0.07},
         "Construction Loans": {"risk": 2.9, "yield" : 0.14},
         "Corp Bonds": {"risk": 1.2, "yield" : 0.1}}

invest_model = LpProblem("my-bond-problem", LpMaximize)

budget = 5000000

X = LpVariable.dict("buy",bonds,0,budget/4)

invest_model += lpSum([X[i] * bonds[i]['yield'] for i in bonds.keys()])

invest_model += lpSum([X[i] * bonds[i]['risk']] for i in bonds.keys()) <= 2 * budget, "risk"
invest_model += lpSum([X[i] for i in bonds.keys()]) <= budget, "budget"
invest_model += X['Gold Stocks'] + X['Platinum Stocks'] >= budget * 0.3, "valuable metals"
invest_model += X['Trade Credits'] + X['Corp Bonds'] >= budget * 0.45, "loans and credits"


print(invest_model)

invest_model.solve()

for i in X.keys():
    print(i, X[i].varValue)

my-bond-problem:
MAXIMIZE
0.14*buy_Construction_Loans + 0.1*buy_Corp_Bonds + 0.19*buy_Gold_Stocks + 0.08*buy_Mortgage_Sec + 0.12*buy_Platinum_Stocks + 0.07*buy_Trade_Credits + 0.0
SUBJECT TO
risk: 2.9 buy_Construction_Loans + 1.2 buy_Corp_Bonds + 3.7 buy_Gold_Stocks
 + 2 buy_Mortgage_Sec + 2.4 buy_Platinum_Stocks + 1.7 buy_Trade_Credits
 <= 10000000

budget: buy_Construction_Loans + buy_Corp_Bonds + buy_Gold_Stocks
 + buy_Mortgage_Sec + buy_Platinum_Stocks + buy_Trade_Credits <= 5000000

valuable_metals: buy_Gold_Stocks + buy_Platinum_Stocks >= 1500000

loans_and_credits: buy_Corp_Bonds + buy_Trade_Credits >= 2250000

VARIABLES
buy_Construction_Loans <= 1250000 Continuous
buy_Corp_Bonds <= 1250000 Continuous
buy_Gold_Stocks <= 1250000 Continuous
buy_Mortgage_Sec <= 1250000 Continuous
buy_Platinum_Stocks <= 1250000 Continuous
buy_Trade_Credits <= 1250000 Continuous

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/py

## Example 6 (Bank Shift Planning)

Hong Kong Bank of Commerce and Industry employs between 10 and 18 tellers depending on the time of day.
* Opening hours 9AM – 5PM
* Full time employees and additional temp workers

Full time employees
* Shift from 9 AM – 5 PM
* 1 hour break (50% at 11, 50% at 12)
* Daily costs $90
* At most 12 in total

Temporal workers
* Work 4 subsequent hours
* Begin at 9, 10, 11, 12 or 1
* Daily cost $110
* Must not cover more than 56 hours


Decision Problem:
* Which type and how many workers should start their shift at what time?
* Objective → Minimize personnel costs

Time Period | Min Num. Tellers
---|---
9to10 | 10
10to11 | 12
11to12 | 14
12to1 | 16
1to2 | 18
2to3| 17
3to4| 15
4to5| 10



We initialize the data to indicate for each demand period what the demand is, whether or not it is fully covered by base workers and which temp shifts will cover it.

In [77]:
demandPeriods = {"9to10":{"demand" : 10, "fulltimeCover": 1, "coverKeys" : ["9to10"]},
                 "10to11":{"demand" : 12, "fulltimeCover": 1, "coverKeys" : ["9to10","10to11"]},
                 "11to12":{"demand" : 14, "fulltimeCover": 0.5, "coverKeys" : ["9to10","10to11","11to12"]},
                 "12to1":{"demand" : 16, "fulltimeCover": 0.5, "coverKeys" : ["9to10","10to11","11to12", "12to1"]},
                 "1to2":{"demand" : 18, "fulltimeCover": 1, "coverKeys" : ["1to2","10to11","11to12", "12to1"]},
                 "2to3":{"demand" : 17, "fulltimeCover": 1, "coverKeys" : ["1to2","11to12", "12to1"]},
                 "3to4":{"demand" : 15, "fulltimeCover": 1, "coverKeys" : ["1to2", "12to1"]},
                 "4to5":{"demand" : 10, "fulltimeCover": 1, "coverKeys" : ["1to2"]},
                 }

Full time staff is single number which we can easily setup:

In [78]:
fullTime = LpVariable("fullTimeStaff",0,12)

Flex workers are created over the first five demand periods. Afterwards it is not sensible to call in worker as they wont have enough hours for the day.

In [79]:
list(demandPeriods.keys())[:5]

['9to10', '10to11', '11to12', '12to1', '1to2']

In [80]:
flexWork = LpVariable.dicts("flex",list(demandPeriods.keys())[:5],0,4)

The final model checks the hourly balance for all time intervals, ensures the hour limit on temp work and minimizes hiring costs.

In [81]:
model = LpProblem('bank_rostering', LpMinimize)

for p in demandPeriods:
  model += fullTime * demandPeriods[p]["fulltimeCover"] + lpSum(flexWork[x] * (x in demandPeriods[p]["coverKeys"]) for x in flexWork) >= demandPeriods[p]["demand"], f"coverage at {p}"

model += lpSum(4*flexWork[x] for x in flexWork) <= 56

model += 90 * fullTime + lpSum(50*flexWork[x] for x in flexWork)

print(model)
model.solve()

bank_rostering:
MINIMIZE
50*flex_10to11 + 50*flex_11to12 + 50*flex_12to1 + 50*flex_1to2 + 50*flex_9to10 + 90*fullTimeStaff + 0
SUBJECT TO
coverage_at_9to10: flex_9to10 + fullTimeStaff >= 10

coverage_at_10to11: flex_10to11 + flex_9to10 + fullTimeStaff >= 12

coverage_at_11to12: flex_10to11 + flex_11to12 + flex_9to10 + 0.5 fullTimeStaff
 >= 14

coverage_at_12to1: flex_10to11 + flex_11to12 + flex_12to1 + flex_9to10
 + 0.5 fullTimeStaff >= 16

coverage_at_1to2: flex_10to11 + flex_11to12 + flex_12to1 + flex_1to2
 + fullTimeStaff >= 18

coverage_at_2to3: flex_11to12 + flex_12to1 + flex_1to2 + fullTimeStaff >= 17

coverage_at_3to4: flex_12to1 + flex_1to2 + fullTimeStaff >= 15

coverage_at_4to5: flex_1to2 + fullTimeStaff >= 10

_C1: 4 flex_10to11 + 4 flex_11to12 + 4 flex_12to1 + 4 flex_1to2 + 4 flex_9to10
 <= 56

VARIABLES
flex_10to11 <= 4 Continuous
flex_11to12 <= 4 Continuous
flex_12to1 <= 4 Continuous
flex_1to2 <= 4 Continuous
flex_9to10 <= 4 Continuous
fullTimeStaff <= 12 Continuous

Welc

1

In [82]:
model.objective.value()

1600.0

In [83]:
fullTime.varValue

10.0

In [84]:
for f in flexWork:
  print(flexWork[f].varValue)

4.0
1.0
4.0
2.0
3.0
